# Autoregressive processes simulation

*Version of this code is running under Windows. Excuted on September 3rd 2019.*
Here we perform time series analysis. We mostly investigate time-series from dynamical systems.
We analyze causal discovery of time series.

We use various causality methods and time-series analysis packages TIGRAMITE (see J.Runge paper)[1]. For other correlation methods applied to time-series check [2]. 

1.  Runge et al, Nature Communications 2015
2.  Donges et al. Unified functional network and nonlinear time series analysis for complex systems science: The pyunicorn package https://arxiv.org/abs/1507.01571

In [1]:
# Imports
import numpy
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
import sklearn

In [ ]:


import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb
from tigramite.models import LinearMediation, Prediction

Consider time series coming from a data generating process.
System $S_1$
\begin{align*}
X^0_t &= 0.7 X^0_{t-1} - 0.8 X^1_{t-1} + \eta^0_t\\
X^1_t &= 0.8 X^1_{t-1} + 0.8 X^3_{t-1} + \eta^1_t\\
X^2_t &= 0.5 X^2_{t-1} + 0.5 X^1_{t-2} + 0.6 X^3_{t-3} + \eta^2_t\\
X^3_t &= 0.7 X^3_{t-1} + \eta^3_t\\
\end{align*}

In system $S_2$ we make the variable $X_3$ independent:...


\begin{align*}
X^0_t &= 0.7 X^0_{t-1} - 0.8 X^1_{t-1} + \eta^0_t\\
X^1_t &= 0.8 X^1_{t-1} + 0.8 X^3_{t-1} + \eta^1_t\\
X^2_t &= 0.5 X^2_{t-1} + 0.5 X^1_{t-2} + 0.6 X^3_{t-3} + \eta^2_t\\
X^3_t &= f(t) + \eta^3_t\\
\end{align*}



where $\eta$ are independent zero-mean unit variance random variables. Our goal is to reconstruct the drivers of each variable. In Tigramite such a process can be generated with the function ``pp.var_process``.

In [ ]:
numpy.random.seed(42)     # Fix random seed
links_coeffs = {0: [((0, -1), 0.7), ((1, -1), -0.8)],
                1: [((1, -1), 0.8), ((3, -1), 0.8)],
                2: [((2, -1), 0.5), ((1, -2), 0.5), ((3, -3), 0.6)],
                3: [((3, -1), 0.7)],
                }
T = 1000     # time series length
data, true_parents_neighbors = pp.var_process(links_coeffs, T=T)
T, N = data.shape

# Initialize dataframe object
dataframe = pp.DataFrame(data)

# Specify time axis and variable names
datatime = numpy.arange(len(data))
var_names = [r'$X^0$', r'$X^1$', r'$X^2$', r'$X^3$']

In [2]:
# System S_2

In [ ]:
numpy.random.seed(42)     # Fix random seed
links_coeffs_2 = {0: [((0, -1), 0.7), ((1, -1), -0.8)],
                1: [((1, -1), 0.8), ((3, -1), 0.8)],
                2: [((2, -1), 0.5), ((1, -2), 0.5), ((3, -3), 0.6)],
                3: [((3, -1), 0.0)],# independent intrance
                }
T = 1000     # time series length
data_2, true_parents_neighbors = pp.var_process(links_coeffs_2, T=T)
T, N = data_2.shape

# Initialize dataframe object #2
dataframe_2 = pp.DataFrame(data_2)

# Specify time axis and variable names
datatime_2 = numpy.arange(len(data_2))
var_names_2 = [r'$X^0$', r'$X^1$', r'$X^2$', r'$X^3$']

First, we plot the time series. This can be done with the function ``tp.plot_timeseries``

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt


'''tp.plot_timeseries(data, datatime, var_names)'''
array = dataframe.values
#print(type(dataframe.values))
#print(array.shape)
print('plotting time series from first dynamical system')
plt.plot(array[:,1])
plt.xlabel('time')

plt.plot(array[:,2])
plt.xlabel('time')

plt.show()


'''tp.plot_timeseries(data, datatime, var_names)'''
array_2 = dataframe_2.values
#print(type(dataframe.values))
#print(array.shape)

print('plotting time series from second dynamical system')
plt.plot(array_2[:,1])
plt.xlabel('time')

plt.plot(array_2[:,3])
plt.xlabel('time')
plt.show()


It's stationary and doesn't contain missing values (covered below). 

After for simplest causality tests we can choose a conditional independence test, here we start with ``ParCorr`` implementing linear partial correlation.